In [ ]:
import os

# pip install python-dotenv
from dotenv import load_dotenv

# pip install openai
from openai import AzureOpenAI

# .env 환경변수 로드
load_dotenv()
AZURE_OAI_ENDPOINT = os.getenv("AZURE_OAI_ENDPOINT")
AZURE_OAI_KEY = os.getenv("AZURE_OAI_KEY")
AZURE_OAI_DEPLOYMENT = os.getenv("AZURE_OAI_DEPLOYMENT")

# AzureOpenAI 객체 생성
client = AzureOpenAI(
    azure_endpoint = AZURE_OAI_ENDPOINT,
    api_key = AZURE_OAI_KEY,
    api_version="2025-01-01-preview"
)

messages = [
            {"role": "system", "content": [{"type": "text", "text": "너는 하인이야. 주인님에게 충성하도록!! (극극존칭 사용)"}]}            
           ]

# 매개변수 Default Value Setting
def request_gpt(input_text, temp=0.7, topP=0.95): 
        
    try: 
        # validation
        if len(input_text) == 0:
            print("Please enter a prompt.")
            return

        print("Sending request for summary to Azure OpenAI endpoint...")
        
        # Add code to send request...
        messages.append({"role": "user", "content": [{"type": "text", "text": input_text}]})

        response = client.chat.completions.create(
            model=AZURE_OAI_DEPLOYMENT,
            messages=messages,
            max_tokens=6553,
            temperature=temp,
            # Top-P : 0~1 사이 값. 모델이 다음 단어 선택 시, 후보 토큰들의 누적 확률값이 임계값 P를 넘는 최소한의 토큰들만 선택
            # Top-K : 0~1 사이 값. 모델이 다음 단어 선택 시, 확률이 높은 순서대로 후보 토큰들의 갯수를 선택.
            # Ex1) 오늘(30%), 내일(25%), 모레(25%), 어제(17%)  ==> 모두 더하면 97%이므로, 후보군이 4개로 설정됨.
            # Ex2) 오늘(30%), 내일(25%), 모레(25%), 우주선(5%), 하늘(4%), 바닥(4%), 물(3%)  ==> 모두 더하면 96%이므로, 후보군이 7개로 설정되지만, 엉뚱한 토큰도 포함됨
            top_p=topP
        )

        # openai.types.chat.chat_completion.ChatCompletion이라는 별도 객체이므로, .으로 속성 접근 가능
        # 일반 dict는 dict명[속성명] 형식으로 접근 가능
        generated_text = response.choices[0].message.content
        messages.append({"role" : "assistant", "content" : generated_text}) # 이전 응답 정보 저장
        print('Answer : ' + generated_text)
            
    except Exception as ex:
        print(ex)

# 현재 파일을 직접 실행한경우, __main__이 출력됨.
# 타 모듈에서 호출한경우, 실행되지 않도록 처리.
if __name__ == '__main__': 
    request_gpt('딸기 설명해줘')
    request_gpt('딸기 설명해줘', 0.1)
    request_gpt('딸기 설명해줘', 0.5, 0.8)
